In [1]:
import utils
inp = utils.get_input(2019, 9)[:-1]
print(inp[:96])

Found cached input for 2019 day 9
1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,3,1,1000,109,988,209,12,9,1000,209


## machine

In [2]:
import operator
from functools import partial
from itertools import repeat, chain
from collections import namedtuple, defaultdict


class IntCode:
    Op = namedtuple('Op', ['func', 'params', 'resolve_last_as_ptr'])

    def __init__(self, program, pointer=0, rel_base=0, **kwargs):
        self._mem = defaultdict(int, enumerate(program))
        self.ptr = pointer
        self.rel = rel_base
        self.state = "started"
        self.output = None
        self._kwargs = kwargs
        self._ops = {
            1: IntCode.Op(partial(IntCode._math, operator.add), 3, True),
            2: IntCode.Op(partial(IntCode._math, operator.mul), 3, True),
            3: IntCode.Op(IntCode._inp, 1, True),
            4: IntCode.Op(IntCode._out, 1, False),
            5: IntCode.Op(partial(IntCode._jump, lambda x: x != 0), 2, False),
            6: IntCode.Op(partial(IntCode._jump, lambda x: x == 0), 2, False),
            7: IntCode.Op(partial(IntCode._math, operator.lt), 3, True),
            8: IntCode.Op(partial(IntCode._math, operator.eq), 3, True),
            9: IntCode.Op(IntCode._base, 1, False),
            99: IntCode.Op(IntCode._halt, 0, False),
        }

    @property
    def mem(self):
        return list(self._mem.values())

    def run(self, inputs=None):
        inputs = iter(inputs or [])

        while self.state != "halted":
            instruction = self._mem[self.ptr]
            opcode, modes = self._opcode(instruction), self._modes(instruction)
            op = self._ops[opcode]
            params_raw = [self._mem[i] for i in range(self.ptr + 1, self.ptr + op.params + 1)]
            params = self.resolve(params_raw, modes, op.resolve_last_as_ptr)

            orig_ptr = self.ptr
            self._mem, self.ptr, self.rel, output, self.state = \
                op.func(self._mem, self.ptr, self.rel, *params, inputs=inputs, **self._kwargs)
            self.output = output if output is not None else self.output

            if self._kwargs.get('_debug', False):
                print(f"{instruction},{','.join(map(str, params_raw))}",
                      f"-> {opcode:2d} {modes} {params}",
                      f"-> {output, self.ptr, self.rel}")

            if (output is not None) or (self.state == "blocked"):
                yield self

            if self.ptr == orig_ptr:
                self.ptr = self.ptr + op.params + 1
        yield self

    def resolve(self, params, modes, resolve_last_as_ptr):
        def _resolve(p, m, as_ptr):
            if int(m) == 0:
                return self._mem[p] if not as_ptr else p
            elif int(m) == 2:
                return self._mem[self.rel + p] if not as_ptr else self.rel + p
            return p
        resolve_as_ptr = chain(repeat(False, len(params) - 1), [resolve_last_as_ptr])
        return list(map(lambda t: _resolve(*t), zip(params, modes, resolve_as_ptr)))

    @staticmethod
    def _opcode(instruction):
        return int(str(instruction)[-2:])

    @staticmethod
    def _modes(instruction):
        return f"{instruction:05d}"[:3][::-1]
    
    # operations
    
    @staticmethod
    def _math(func, mem, ptr, rel, a, b, out, **kwargs):
        mem[out] = int(func(a, b))
        return mem, ptr, rel, None, "running"

    @staticmethod
    def _inp(mem, ptr, rel, out, inputs, **kwargs):
        try:
            mem[out] = int(next(inputs))
            return mem, ptr, rel, None, "running"
        except StopIteration:
            return mem, ptr, rel, None, "blocked"

    @staticmethod
    def _out(mem, ptr, rel, val, **kwargs):
        return mem, ptr, rel, val, "running"

    @staticmethod
    def _jump(func, mem, ptr, rel, cond, val, **kwargs):
        return mem, (val if func(cond) else ptr), rel, None, "running"

    @staticmethod
    def _base(mem, ptr, rel, val, **kwargs):
        return mem, ptr, (rel + val), None, "running"
    
    @staticmethod
    def _halt(mem, ptr, rel, *args, **kwargs):
        return mem, ptr, rel, None, "halted"


In [3]:
def execute(program, inputs):
    vm = IntCode(program, _debug=False)
    vm_gen = vm.run(inputs)
    while True:
        vm = next(vm_gen)
        if vm.state == "halted":
            break
    return vm.output


## part 1

In [4]:
def part_1(program):
    return execute(program, [1])


program = list(map(int, inp.split(',')))
print(part_1(program))

3765554916


## part 2

In [5]:
def part_2(program):
    return execute(program, [2])


program = list(map(int, inp.split(',')))
print(part_2(program))

76642
